# Flu Shot Learning

The goal of this notebook is to predict the whether each of a group of individuals have received two types of flu vaccines using a number of different relevant features.

This project is part of the [Flu Shot Learning competition](https://www.drivendata.org/competitions/66/flu-shot-learning/) by Driven Data.

The data for this competition comes from the National 2009 H1N1 Flu Survey (NHFS).  In their own words:

>The National 2009 H1N1 Flu Survey (NHFS) was sponsored by the National Center for Immunization and Respiratory Diseases (NCIRD) and conducted jointly by NCIRD and the National Center for Health Statistics (NCHS), Centers for Disease Control and Prevention (CDC). The NHFS was a list-assisted random-digit-dialing telephone survey of households, designed to monitor influenza immunization coverage in the 2009-10 season.
>
>The target population for the NHFS was all persons 6 months or older living in the United States at the time of the interview. Data from the NHFS were used to produce timely estimates of vaccination coverage rates for both the monovalent pH1N1 and trivalent seasonal influenza vaccines.
     
[<cite>National 2009 H1N1 Flu Survey (NHFS) Public-Use Data File (PUF)</cite>](https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/nis/nhfs/nhfspuf_readme.txt)

In [1]:
# Import basic libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set_style('dark')

# Read data

Training data are stored in 2 separate files, one with the features and one with the labels for each entry.

In [2]:
def get_data(feat_path, label_path=''):
    df_X = pd.read_csv(feat_path)
    df_X.set_index('respondent_id', inplace=True)
    nominals = [col for col in df_X.columns if df_X[col].nunique() == 2] + [
        'race', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation']
    ordinals = [col for col in df_X.columns if col not in nominals]
    age_cat = pd.api.types.CategoricalDtype(['18 - 34 Years', '35 - 44 Years', '45 - 54 Years', '55 - 64 Years', '65+ Years'],
                                           ordered=True)
    college_cat = pd.api.types.CategoricalDtype(['< 12 Years', '12 Years', 'Some College', 'College Graduate'], ordered=True)
    income_cat = pd.api.types.CategoricalDtype(['Below Poverty', '<= $75,000, Above Poverty', '> $75,000'], ordered=True)
    df_X['age_group'] = df_X['age_group'].astype(age_cat)
    df_X['education'] = df_X['education'].astype(college_cat)
    df_X['income_poverty'] = df_X['income_poverty'].astype(income_cat)
    for col in ordinals:
        if col not in ['age_group', 'education', 'income_poverty']:
            df_X[col] = df_X[col].astype(pd.api.types.CategoricalDtype([1., 2., 3., 4., 5.], ordered=True))
    for col in nominals:
        df_X[col] = df_X[col].astype('category')
    ret = [df_X]
    if label_path != '':
        df_Y = pd.read_csv(label_path)
        df_Y.set_index('respondent_id', inplace=True)
        df_Y = df_Y.astype(bool)
        ret.append(df_Y)
    return ret

In [3]:
X, Y = get_data('data/training_set_features.csv', 'data/training_set_labels.csv')

In [4]:
X

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,NaN,NaN,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",NaN,NaN,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,NaN,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",NaN,NaN,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,NaN,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,NaN,NaN,NaN,NaN
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,NaN,fcxhlnwr,cmhcxjea
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",NaN,NaN,NaN,NaN


In [5]:
Y

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
0,False,False
1,False,True
2,False,False
3,False,True
4,False,False
...,...,...
26702,False,False
26703,False,False
26704,False,True


In [6]:
flushot_df = pd.concat([X, Y], axis='columns')
flushot_df.sample(5)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,
6528,3.0,NaN,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,NaN,NaN,NaN,False,True
4129,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,...,Rent,Employed,oxchjgsf,"MSA, Not Principle City",1.0,3.0,NaN,xtkaffoo,False,False
16632,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,Own,Employed,lzgpxyit,"MSA, Not Principle City",1.0,NaN,fcxhlnwr,cmhcxjea,False,False
15568,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,Own,Not in Labor Force,dqpwygqj,"MSA, Not Principle City",1.0,1.0,NaN,NaN,False,False
4529,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,Own,Employed,mlyzmhmf,"MSA, Not Principle City",NaN,NaN,xqicxuve,mxkfnird,True,True


# Exploratory data analysis -- Feature engineering

In [7]:
flushot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   h1n1_concern                 23319 non-null  category
 1   h1n1_knowledge               24085 non-null  category
 2   behavioral_antiviral_meds    26636 non-null  category
 3   behavioral_avoidance         26499 non-null  category
 4   behavioral_face_mask         26688 non-null  category
 5   behavioral_wash_hands        26665 non-null  category
 6   behavioral_large_gatherings  26620 non-null  category
 7   behavioral_outside_home      26625 non-null  category
 8   behavioral_touch_face        26579 non-null  category
 9   doctor_recc_h1n1             24547 non-null  category
 10  doctor_recc_seasonal         24547 non-null  category
 11  chronic_med_condition        25736 non-null  category
 12  child_under_6_months         25887 non-null  category
 13  h

In [8]:
flushot_df.nunique()

h1n1_concern                    3
h1n1_knowledge                  2
behavioral_antiviral_meds       2
behavioral_avoidance            2
behavioral_face_mask            2
behavioral_wash_hands           2
behavioral_large_gatherings     2
behavioral_outside_home         2
behavioral_touch_face           2
doctor_recc_h1n1                2
doctor_recc_seasonal            2
chronic_med_condition           2
child_under_6_months            2
health_worker                   2
health_insurance                2
opinion_h1n1_vacc_effective     5
opinion_h1n1_risk               5
opinion_h1n1_sick_from_vacc     5
opinion_seas_vacc_effective     5
opinion_seas_risk               5
opinion_seas_sick_from_vacc     5
age_group                       5
education                       4
race                            4
sex                             2
income_poverty                  3
marital_status                  2
rent_or_own                     2
employment_status               3
hhs_geo_region

In [9]:
for col in flushot_df.columns:
    if flushot_df[col].nunique() > 2:
        display(flushot_df[col].value_counts())

2.0    10575
1.0     8153
3.0     4591
4.0        0
5.0        0
Name: h1n1_concern, dtype: int64

4.0    11683
5.0     7166
3.0     4723
2.0     1858
1.0      886
Name: opinion_h1n1_vacc_effective, dtype: int64

2.0    9919
1.0    8139
4.0    5394
5.0    1750
3.0    1117
Name: opinion_h1n1_risk, dtype: int64

2.0    9129
1.0    8998
4.0    5850
5.0    2187
3.0     148
Name: opinion_h1n1_sick_from_vacc, dtype: int64

4.0    11629
5.0     9973
2.0     2206
1.0     1221
3.0     1216
Name: opinion_seas_vacc_effective, dtype: int64

...

MSA, Not Principle  City    11645
MSA, Principle City          7864
Non-MSA                      7198
Name: census_msa, dtype: int64

1.0    14474
2.0     2803
3.0     1125
4.0        0
5.0        0
Name: household_adults, dtype: int64

1.0    3175
2.0    2864
3.0    1747
4.0       0
5.0       0
Name: household_children, dtype: int64

fcxhlnwr    2468
wxleyezf    1804
ldnlellj    1231
pxcmvdjn    1037
atmlpfrs     926
arjwrbjb     871
xicduogh     851
mfikgejo     614
vjjrobsf     527
rucpziij     523
xqicxuve     511
saaquncn     338
cfqqtusy     325
nduyfdeo     286
mcubkhph     275
wlfvacwt     215
dotnnunm     201
haxffmxo     148
msuufmds     124
phxvnwax      89
qnlwzans      13
Name: employment_industry, dtype: int64

xtkaffoo    1778
mxkfnird    1509
emcorrxb    1270
cmhcxjea    1247
xgwztkwe    1082
hfxkjkmi     766
qxajmpny     548
xqwwgdyp     485
kldqjyjy     469
uqqtjvyb     452
tfqavkke     388
ukymxvdu     372
vlluhbov     354
oijqvulv     344
ccgxvspp     341
bxpfxfdn     331
haliazsg     296
rcertsgn     276
xzmlyyjv     248
dlvbwzss     227
hodpvpew     208
dcjcmpih     148
pvmttkik      98
Name: employment_occupation, dtype: int64

In [10]:
for col in flushot_df.columns:
    if flushot_df[col].nunique() == 2:
        display(flushot_df[col].value_counts())

1.0    14598
2.0     9487
3.0        0
4.0        0
5.0        0
Name: h1n1_knowledge, dtype: int64

0.0    25335
1.0     1301
Name: behavioral_antiviral_meds, dtype: int64

1.0    19228
0.0     7271
Name: behavioral_avoidance, dtype: int64

0.0    24847
1.0     1841
Name: behavioral_face_mask, dtype: int64

1.0    22015
0.0     4650
Name: behavioral_wash_hands, dtype: int64

...

Female    15858
Male      10849
Name: sex, dtype: int64

Married        13555
Not Married    11744
Name: marital_status, dtype: int64

Own     18736
Rent     5929
Name: rent_or_own, dtype: int64

False    21033
True      5674
Name: h1n1_vaccine, dtype: int64

False    14272
True     12435
Name: seasonal_vaccine, dtype: int64

### Observations
1. There are several missing values
2. All features are categorical
3. Features with cardinality greater than 2, except `race`, `employment_status`, `hhs_geo_region`, `census_msa`, `employment_industry` and `employment_occupation` are ordinal categoricals.  All binary features, and `race`, `employment_status`, `hhs_geo_region`, `census_msa`, `employment_industry` and `employment_occupation` are nominal categoricals.